In [ ]:
from pathlib import Path
import pandas as pd

file_path = Path("data/cases_texts_127_2.json")
data_list = []

with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            
            data = json.loads(line)
            data_list.append(data)
        except json.JSONDecodeError as e:
            print(f"Ошибка декодирования JSON: {e}")

df = pd.DataFrame(data_list)
df.head(5)


,id,region,instance,name,caseNumber,entryDate,names,judge,resultDate,decision,endDate,steps,sides,people,consideredBy,text,connectedLinks,links,cui
0,0,1,FIRST,Теучежский районный суд Республики Адыгея,1-63/2018,10.08.2018,Абкелямов Эльдар Курсатович - ст.163 ч.3 п.а; ...,Бжассо Светлана Теучежевна,03.09.2018,Направлено ПО ПОДСУДНОСТИ (подведомственности),None,[{'eventName': 'Регистрация поступившего в суд...,[],"[{'name': 'Абкелямов Эльдар Курсатович', 'arti...",None,[],{},{'META': 'http://teuchezhsky--adg.sudrf.ru/mod...,01RS0007-01-2013-000001-40
1,1,1,FIRST,Теучежский районный суд Республики Адыгея,1-1/2017 (1-1/2016; 1-1/2015; 1-1/2014; 1-91/2...,26.11.2013,Абкелямов Эльдар Курсатович - ст.163 ч.3 п.а; ...,Шумен Байзет Азметович,25.11.2017,Вынесен ПРИГОВОР,ОТМЕНЕН,[{'eventName': 'Регистрация поступившего в суд...,[],"[{'name': 'Абкелямов Эльдар Курсатович', 'arti...",None,[],{},{'META': 'http://teuchezhsky--adg.sudrf.ru/mod...,01RS0007-01-2013-000001-40
2,2,1,FIRST,Майкопский районный суд Республики Адыгея,1-2/2019 (1-112/2018;),04.10.2018,Арутюнов Габриэль Эдуардович - ст.163 ч.3 п.а;...,Дарзиян Альберт Артюшевич,27.12.2019,Вынесен ПРИГОВОР,04.02.2020,[{'eventName': 'Регистрация поступившего в суд...,"[{'sideType': 'Защитник (адвокат)', 'sideName'...","[{'name': 'Арутюнов Габриэль Эдуардович', 'art...",Рассмотрено единолично судьей,[],{},{'META': 'http://maikopskyr--adg.sudrf.ru/modu...,01RS0005-01-2018-001250-68
3,3,1,FIRST,ВЕРХОВНЫЙ СУД РЕСПУБЛИКИ АДЫГЕЯ,2-4/2018,27.08.2018,Волков Владимир Иванович - ст.159 ч.4; ст.127....,Войстриков Михаил Николаевич,13.09.2018,Направлено ПО ПОДСУДНОСТИ (подведомственности),None,[],[],[],None,[],{},{'META': 'http://vs--adg.sudrf.ru/modules.php?...,None
4,4,22,FIRST,Ключевский районный суд Алтайского края,1-2/2012 (1-91/2011;),09.12.2011,Лебедев Владимир Владимирович - ст.163 ч.2 п.в...,Шевченко В.И,26.10.2012,Вынесен ПРИГОВОР,28.02.2013,[{'eventName': 'Регистрация поступившего в суд...,"[{'sideType': 'Защитник (адвокат)', 'sideName'...","[{'name': 'Лебедев Владимир Владимирович', 'ar...",Рассмотрено единолично судьей,[],{},{'META': 'http://kluchevsky--alt.sudrf.ru/modu...,22RS0024-01-2011-000405-86


In [ ]:
#Сохраним открытые тексты приговоров в txt для анализа 
with open('texts_127_2.txt', 'w', encoding='utf-8') as f:
  
    for index, row in df_unique.iterrows():
        # Проверяем, что поле 'text' не пустое
        if pd.notna(row['text']):
            line = f"caseNumber: {row['caseNumber']}, name: {row['name']} text: {row['text']}\n"
            f.write(line)


Теперь оставим только обвинительные приговоры в первой инстанции, вынесенные начиная с 2010 года

In [109]:
import pandas as pd
df["entryDate"] = pd.to_datetime(df["entryDate"], format="%d.%m.%Y")
df = df[df["entryDate"] >= "2010-01-01"]


In [110]:
df_filtered = df.loc[df.sort_values(by="text", na_position="last").duplicated(subset="caseNumber", keep="last") == False]
df_filtered["instance"] = "FIRST"
df_filtered = df_filtered[
    df_filtered['decision'].str.contains(r'Вынесен ПРИГОВОР', case=False, na=False)
]

df_filtered = df_filtered[~df_filtered['endDate'].str.contains('ОТМЕНЕН', case=False, na=False)]
df_filtered = df_filtered[~df_filtered['endDate'].str.contains('ОБЖАЛУЕТСЯ', case=False, na=False)]
len(df_filtered)


C:\Users\1\AppData\Local\Temp/ipykernel_11492/2476022938.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["instance"] = "FIRST"


53

In [ ]:
df_filtered.to_csv("127_2_cases.csv", index=False, encoding="utf-8")
